In [1]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [128]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [28]:
if socket.gethostname() == 'Rohits-MacBook-Pro.local':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/'
else: 
    rootdir = '' # Enter your hone dir here

In [29]:
client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='')
client._request_signer.sign = (lambda *args, **kwargs: None)

resource = boto3.resource('s3')
resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)

### Checking file size within bucket 
*bucket name: cse-cic-ids2018 ref:https://www.unb.ca/cic/datasets/ids-2018.html*

In [30]:
cse_bucket = 'cse-cic-ids2018'
resp = client.list_objects_v2(Bucket=cse_bucket)

In [7]:
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    if obj["Size"] > 0:
        print(f'key:{key}, size: {round(size/1024/1024,2)} mb')

key:Original Network Traffic and Log data/Friday-02-03-2018/logs.zip, size: 225.77 mb
key:Original Network Traffic and Log data/Friday-02-03-2018/pcap.zip, size: 42714.92 mb
key:Original Network Traffic and Log data/Friday-16-02-2018/logs.zip, size: 148.05 mb
key:Original Network Traffic and Log data/Friday-16-02-2018/pcap.zip, size: 36750.48 mb
key:Original Network Traffic and Log data/Friday-23-02-2018/logs.zip, size: 199.79 mb
key:Original Network Traffic and Log data/Friday-23-02-2018/pcap.zip, size: 56356.36 mb
key:Original Network Traffic and Log data/Thursday-01-03-2018/logs.zip, size: 217.08 mb
key:Original Network Traffic and Log data/Thursday-01-03-2018/pcap.zip, size: 49932.55 mb
key:Original Network Traffic and Log data/Thursday-15-02-2018/logs.zip, size: 142.58 mb
key:Original Network Traffic and Log data/Thursday-15-02-2018/pcap.zip, size: 39370.9 mb
key:Original Network Traffic and Log data/Thursday-22-02-2018/logs.zip, size: 195.28 mb
key:Original Network Traffic and Lo

### Looks like _Processed Traffic Data for ML Algorithms_ folder has the necessary data we need.
####  If you do it on local this will take a while.

In [8]:
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    file_name = key.split('/')[1]
    if size > 0 and 'Processed Traffic' in key:
        print(f'Downloading key:{key}, size: {round(size/1024/1024,2)} mb')
        resource.meta.client.download_file(cse_bucket, key, rootdir+file_name)
        print(f'{file_name} downloaded')

Friday-02-03-2018_TrafficForML_CICFlowMeter.csv downloaded
Friday-16-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Friday-23-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv downloaded
Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv downloaded


### [Work in progress]Analyzing Data

In [71]:
df_traffic = pd.read_csv(f'{rootdir}Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')

In [73]:
dtypes = df_traffic.dtypes
print(f'Number of features {type(dtypes)}')

Number of features <class 'pandas.core.series.Series'>


In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dtypes)

Dst Port               int64
Protocol               int64
Timestamp             object
Flow Duration          int64
Tot Fwd Pkts           int64
Tot Bwd Pkts           int64
TotLen Fwd Pkts        int64
TotLen Bwd Pkts      float64
Fwd Pkt Len Max        int64
Fwd Pkt Len Min        int64
Fwd Pkt Len Mean     float64
Fwd Pkt Len Std      float64
Bwd Pkt Len Max        int64
Bwd Pkt Len Min        int64
Bwd Pkt Len Mean     float64
Bwd Pkt Len Std      float64
Flow Byts/s          float64
Flow Pkts/s          float64
Flow IAT Mean        float64
Flow IAT Std         float64
Flow IAT Max         float64
Flow IAT Min         float64
Fwd IAT Tot          float64
Fwd IAT Mean         float64
Fwd IAT Std          float64
Fwd IAT Max          float64
Fwd IAT Min          float64
Bwd IAT Tot          float64
Bwd IAT Mean         float64
Bwd IAT Std          float64
Bwd IAT Max          float64
Bwd IAT Min          float64
Fwd PSH Flags          int64
Bwd PSH Flags          int64
Fwd URG Flags 

In [24]:
df_traffic.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,...,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000,1048575.00000
mean,8423.42940,8.04985,11514208.72033,5.88728,6.98318,392.08376,5271.27532,210.43115,8.24920,54.61640,...,2.04423,17.45403,122957.85345,61309.72034,190523.05514,83054.45673,4781197.35081,106422.42249,4863777.33665,4677016.56342
std,15161.75671,4.43263,30121132.47167,89.03317,212.95795,2159.77937,308876.83187,266.94967,20.94972,57.62840,...,13.92463,5.17953,2083098.85479,1261694.00443,2761455.52967,1759831.61262,17413075.11403,1462605.49005,17568079.17726,17333032.00493
min,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,80.00000,6.00000,516.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
50%,3389.00000,6.00000,11242.00000,3.00000,1.00000,63.00000,129.00000,46.00000,0.00000,40.00000,...,1.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
75%,8080.00000,6.00000,2210150.00000,7.00000,5.00000,365.00000,582.00000,326.00000,0.00000,108.66667,...,3.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
max,65534.00000,17.00000,119999997.00000,43159.00000,69241.00000,1100627.00000,101000000.00000,1711.00000,1460.00000,1460.00000,...,9262.00000,44.00000,111000000.00000,74900000.00000,111000000.00000,111000000.00000,120000000.00000,75900000.00000,120000000.00000,120000000.00000


In [38]:
df_traffic.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


### Most of the Data Files have header records within data file so we need to clean them up as they are causing issues in schema detection

In [127]:
dest = rootdir+'data_parquet/'
tables = {}
for obj in resp['Contents']:
    size = obj["Size"]
    key = obj["Key"]
    file_name = key.split('/')[1]
    dest_name = file_name.split('.')[0]+'_clean.parquet'
    
    if size > 0 and 'Processed Traffic' in key:
        print(f'Processing file_name:{file_name}')
        df_temp = pd.read_csv(f'{rootdir}{file_name}')
        #print(df_temp.dtypes)
        df_temp = df_temp[df_temp['Dst Port'] != 'Dst Port'] #Cleaning up necessary headers within rows
        df_temp.to_csv(f'{rootdir}clean_{file_name}',encoding='utf-8', index=False)
        df_temp = pd.read_csv(f'{rootdir}clean_{file_name}')
        tables[file_name] = pa.Table.from_pandas(df_temp)
        pq.write_table(tables[file_name], f'{dest}{dest_name}', compression='snappy')
        print(f'Finished Parquet conversion for file_name:{file_name}')
        

Processing file_name:Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv


/var/folders/lb/z26czmjn11z59vv2p8rgj_5h0000gn/T/ipykernel_13197/1456713873.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f'{rootdir}{file_name}')


Finished Parquet conversion for file_name:Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Finished Parquet conversion for file_name:Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Processing file_name:Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


/var/folders/lb/z26czmjn11z59vv2p8rgj_5h0000gn/T/ipykernel_13197/1456713873.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(f'{rootdir}{file_name}')


Finished Parquet conversion for file_name:Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


### Checking sizes csv vs parquet:

In [146]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
type(list_of_files)
files_with_size = [ (file_path, os.stat(file_path).st_size) for file_path in list_of_files ]
files_with_size.sort()
for file_path, file_size in files_with_size:
    if 'clean_' in file_path:
        file_name = file_path.split('/')
        print(f'Size_MB: {round(file_size/1024/1024,2)}, Data File: {file_name[-1]}')

Size_MB: 387.13, Data File: clean_Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 349.38, Data File: clean_Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 391.94, Data File: clean_Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 4051.86, Data File: clean_Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 110.12, Data File: clean_Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 385.87, Data File: clean_Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 391.71, Data File: clean_Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 371.47, Data File: clean_Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 344.18, Data File: clean_Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Size_MB: 209.53, Data File: clean_Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


In [147]:
list_of_files = filter( os.path.isfile, glob.glob(dest + '*') )
type(list_of_files)
files_with_size = [ (file_path, os.stat(file_path).st_size) for file_path in list_of_files ]
files_with_size.sort()
for file_path, file_size in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')
        print(f'Size_MB: {round(file_size/1024/1024,2)}, Data File: {file_name[-1]}')

Size_MB: 109.63, Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 92.82, Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 130.97, Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 1099.3, Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 36.41, Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 129.17, Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 130.86, Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 97.72, Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 61.5, Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Size_MB: 70.36, Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet


# Reading data from Parquet

In [140]:
dfs_parquet = {}
for file_path, file_size in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')
        

Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet


In [145]:
for k,df in dfs_parquet.items():
    print(f'DF:{k}, Shape{df.shape}')
    print(f'Schema:')
    print(df.info())

DF:Friday-02-03-2018, Shape(1048575, 80)
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  float64
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-nu

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613071 entries, 0 to 613070
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           613071 non-null  int64  
 1   Protocol           613071 non-null  int64  
 2   Timestamp          613071 non-null  object 
 3   Flow Duration      613071 non-null  int64  
 4   Tot Fwd Pkts       613071 non-null  int64  
 5   Tot Bwd Pkts       613071 non-null  int64  
 6   TotLen Fwd Pkts    613071 non-null  int64  
 7   TotLen Bwd Pkts    613071 non-null  int64  
 8   Fwd Pkt Len Max    613071 non-null  int64  
 9   Fwd Pkt Len Min    613071 non-null  int64  
 10  Fwd Pkt Len Mean   613071 non-null  float64
 11  Fwd Pkt Len Std    613071 non-null  float64
 12  Bwd Pkt Len Max    613071 non-null  int64  
 13  Bwd Pkt Len Min    613071 non-null  int64  
 14  Bwd Pkt Len Mean   613071 non-null  float64
 15  Bwd Pkt Len Std    613071 non-null  float64
 16  Fl

In [148]:
for k,df in dfs_parquet.items():
    print(f'DF:{k}, Shape{df.shape}')

DF:Friday-02-03-2018, Shape(1048575, 80)
DF:Friday-16-02-2018, Shape(1048574, 80)
DF:Friday-23-02-2018, Shape(1048575, 80)
DF:Thuesday-20-02-2018, Shape(7948748, 84)
DF:Thursday-01-03-2018, Shape(331100, 80)
DF:Thursday-15-02-2018, Shape(1048575, 80)
DF:Thursday-22-02-2018, Shape(1048575, 80)
DF:Wednesday-14-02-2018, Shape(1048575, 80)
DF:Wednesday-21-02-2018, Shape(1048575, 80)
DF:Wednesday-28-02-2018, Shape(613071, 80)
